In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense



In [15]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import pandas as pd

In [20]:
!pip install chardet

In [22]:
import chardet

with open("/content/judge_tweet_product_company.csv", 'rb') as f:
    encoding = chardet.detect(f.read())['encoding']

data = pd.read_csv("/content/judge_tweet_product_company.csv", encoding=encoding)

In [23]:
data = pd.read_csv("/content/judge_tweet_product_company.csv", encoding='latin-1')

In [25]:
print(data.head())

                                          tweet_text  \
0  .@wesley83 I have a 3G iPhone. After 3 hrs twe...   
1  @jessedee Know about @fludapp ? Awesome iPad/i...   
2  @swonderlin Can not wait for #iPad 2 also. The...   
3  @sxsw I hope this year's festival isn't as cra...   
4  @sxtxstate great stuff on Fri #SXSW: Marissa M...   

  emotion_in_tweet_is_directed_at  \
0                          iPhone   
1              iPad or iPhone App   
2                            iPad   
3              iPad or iPhone App   
4                          Google   

  is_there_an_emotion_directed_at_a_brand_or_product  
0                                   Negative emotion  
1                                   Positive emotion  
2                                   Positive emotion  
3                                   Negative emotion  
4                                   Positive emotion  


In [29]:
# Check the column names of the DataFrame
print(df.columns)

# View the first few rows of the DataFrame
print(df.head())

# Drop the column if it exists
if 'emotion_in_tweet_is_directed_at' in df.columns:
    df = df.drop(columns=['emotion_in_tweet_is_directed_at'])
else:
    print("The column 'emotion_in_tweet_is_directed_at' does not exist in the DataFrame.")

Index(['tweet_text', 'is_there_an_emotion_directed_at_a_brand_or_product'], dtype='object')
                                          tweet_text  \
0  .@wesley83 I have a 3G iPhone. After 3 hrs twe...   
1  @jessedee Know about @fludapp ? Awesome iPad/i...   
2  @swonderlin Can not wait for #iPad 2 also. The...   
3  @sxsw I hope this year's festival isn't as cra...   
4  @sxtxstate great stuff on Fri #SXSW: Marissa M...   

  is_there_an_emotion_directed_at_a_brand_or_product  
0                                   Negative emotion  
1                                   Positive emotion  
2                                   Positive emotion  
3                                   Negative emotion  
4                                   Positive emotion  
The column 'emotion_in_tweet_is_directed_at' does not exist in the DataFrame.


In [33]:
data.columns

Index(['tweet_text', 'emotion_in_tweet_is_directed_at',
       'is_there_an_emotion_directed_at_a_brand_or_product'],
      dtype='object')

In [35]:
df = df[['tweet_text', 'is_there_an_emotion_directed_at_a_brand_or_product']]


In [37]:
# Remove rows with missing values
df = df.dropna()


In [38]:
# Preprocess the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['tweet_text'])
sequences = tokenizer.texts_to_sequences(df['tweet_text'])
word_index = tokenizer.word_index
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)


In [40]:
# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['is_there_an_emotion_directed_at_a_brand_or_product'])


In [42]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['label'], test_size=0.2, random_state=42)


In [44]:
# Build the model
model = Sequential()
model.add(Embedding(len(word_index) + 1, 100, input_length=max_sequence_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))


In [46]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [47]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)


Epoch 1/10
91/91 [==============================] - 12s 103ms/step - loss: -15.5490 - accuracy: 0.0617 - val_loss: -25.4102 - val_accuracy: 0.0591
Epoch 2/10
91/91 [==============================] - 8s 86ms/step - loss: -31.6376 - accuracy: 0.0624 - val_loss: -37.5580 - val_accuracy: 0.0591
Epoch 3/10
91/91 [==============================] - 9s 98ms/step - loss: -43.5951 - accuracy: 0.0624 - val_loss: -49.2316 - val_accuracy: 0.0591
Epoch 4/10
91/91 [==============================] - 9s 95ms/step - loss: -55.3646 - accuracy: 0.0624 - val_loss: -61.1501 - val_accuracy: 0.0591
Epoch 5/10
91/91 [==============================] - 8s 85ms/step - loss: -67.2115 - accuracy: 0.0624 - val_loss: -72.7129 - val_accuracy: 0.0591
Epoch 6/10
91/91 [==============================] - 9s 98ms/step - loss: -78.7278 - accuracy: 0.0624 - val_loss: -84.1108 - val_accuracy: 0.0591
Epoch 7/10
91/91 [==============================] - 9s 101ms/step - loss: -90.1269 - accuracy: 0.0624 - val_loss: -95.3967 - val

In [49]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


57/57 [==============================] - 1s 17ms/step - loss: -130.3331 - accuracy: 0.0665
Test Loss: -130.3330841064453
Test Accuracy: 0.06652006506919861
